# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd
import os
import this


os.listdir()

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


['main.ipynb', '.ipynb_checkpoints']

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [6]:
# Your code here:
pokemon["Legendary"].value_counts()

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [33]:
# Your code here:
# Means (Mu's)
poke_means = pokemon.groupby(["Legendary"])["Total"].mean()

# Standard Deviations (Sigma's)
poke_sigmas = pokemon.groupby(["Legendary"])["Total"].std()

# Presenting the output to the User
print("The Total points mean of the Pokemon are:\n", poke_means)
print("\nThe Total poins standard deviation of the Pokemon are:\n", poke_sigmas)

The Total points mean of the Pokemon are:
 Legendary
False    417.213605
True     637.384615
Name: Total, dtype: float64

The Total poins standard deviation of the Pokemon are:
 Legendary
False    106.760417
True      60.937389
Name: Total, dtype: float64


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [42]:
# Your code here:
# importing ttest_ind method 
from scipy.stats import ttest_ind

# declaring the independent samples:
legendary = pokemon[pokemon["Legendary"] == True]
non_legendary = pokemon[pokemon["Legendary"] == False]

# checking ——> O.K.
len(legendary)
len(non_legendary)

# t-testing:
ttest_ind(legendary["Total"], non_legendary["Total"], equal_var = False)

Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957446e-47)

What do you conclude from this test? Write your conclusions below.

In [6]:
# Your conclusions here:
"""
From: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

«We can use this test, if we observe two independent samples from the same or different 
population(...). The test measures whether the average (expected) value differs significantly 
across samples. If we observe a large p-value, for example larger than 0.05 or 0.1, then we 
*cannot* reject the null hypothesis of identical average scores. If the p-value is smaller 
than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.»

Since we didn't predefine any treshold, we *shouldn't* actually conclude anything. That being
said, since the p-value is incredibly low, we can say that we *could* reject the null 
hypothesis, i.e, we could reject the idea that the average of each sample (legendary and 
non-legendary pokemons) is identical.
"""

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [46]:
# Your code here:
pokemon["Type 1"].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Rock         44
Electric     44
Dragon       32
Ghost        32
Ground       32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [110]:
# Your code here:
# It's not specified in this case, but I think that its meant to be the "Total" as well:
# True for "Water", "False" for all other!! :D
water_vs_other = pokemon.pivot_table(index = pokemon["Type 1"] == "Water", values = "Total", 
                    aggfunc = ["mean", "std"])

water_vs_other

,mean,std
,Total,Total
Type 1,,
False,435.859012,121.091682
True,430.455357,113.188266


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [113]:
# Your code here:
# this?
# declaring the independent samples:
type1_water = pokemon[pokemon["Type 1"] == "Water"]
others_type1 = pokemon[pokemon["Type 1"] != "Water"]

# t-testing:
ttest_ind(type1_water["Total"], others_type1["Total"], equal_var = True)

Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [10]:
# Your conclusions here:
"""
Applying the same logic as above, in this case we *could NOT* reject the null hypothesis 
(assuming we had establish a threshold equal to or bellow 65%).

(Or we could simply say that there isn't much difference in the expectade values (mean) of the 
two samples)
"""

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [118]:
# Your code here:
from scipy.stats import ttest_rel

# this?
ttest_rel(pokemon["Attack"], pokemon["Defense"], axis=0, nan_policy='propagate')

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [12]:
# Your conclusions here:
"""
From: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html?highlight=ttest_rel#scipy.stats.ttest_rel

«The test measures whether the average score differs significantly across samples (e.g. exams).
If we observe a large p-value, for example greater than 0.05 or 0.1 then we cannot reject the 
null hypothesis of identical average scores. If the p-value is smaller than the threshold, 
e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages. Small p-values are 
associated with large t-statistics.

Using the same logic as before, we can ONLY say that we *could* reject our null hypothesis, 
for the p-value is very small. We're still missing our predefined threshold to properly do it.
"""

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [121]:
# Your code here:
# this?
ttest_rel(pokemon["Sp. Atk"], pokemon["Sp. Def"], axis=0, nan_policy='propagate')

Ttest_relResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

Describe the results of the test in the cell below.

In [14]:
# Your conclusions here:
"""
Using the same logic as before, win this case it would seem to be less reasonable to reject
the null hypothesis, as the p-value seems somewhat significant (aprox 39.34%). But we really
should have a predefined threshold BEDORE doing this evaluation.
"""

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [122]:
# Your code here:
from scipy.stats import ttest_1samp

# this? (now I'm confused)
ttest_1samp(pokemon[["Defense", "Attack"]], 0, nan_policy = "propagate")

Ttest_1sampResult(statistic=array([66.97712772, 68.84393493]), pvalue=array([0., 0.]))

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [17]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [18]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [19]:
# Your answer here:

